In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import re
from collections import Counter
from IPython.display import display, HTML


# Exploratory Data Analysis of Nutrient Consumption and Eating Habbits 

## Author: Tsvetan Dimitrov

### Abstract
The goal of this analysis is to detect what are actually people eating, how much of it (broken down in its nutrient content) and what food preferences do they have and under which circumstances do they make nutritional checks or how often they cook etc. The first dataset we are going to use is the "Open Food Facts" dataset where we are going to detect how much sugar, carbohydrates, salt, sodium, protein, saturated fat and total fat are the different countries consuming per 100g of food. Next we will use a dataset called "Food choices and preferences of college students" where we will analyse eating habbits and food preferences of college student categorized by gender, income, sports activity, etc. Our third dataset contains nutrition facts about McDonalds menu, which is one of the most popular types of food available worldwide.

### 1. Open Food Facts
This data does not show us what people actually eat on any sort of basis, but it does show us what may be readily available to each user who uploads information. To accomplish the task set out above, we will aggregate the data by country and take mean values for sugar, carbohydrates, salt, sodium, protein, saturated fat and total fat per 100g as well as the number of entries used to calculate that mean. First we are going to read the dataset:

In [3]:
food_facts = pd.read_csv('data/en.openfoodfacts.org.products.tsv', low_memory=False, sep='\t')
pd.set_option("display.max_columns", len(food_facts.columns))
food_facts.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, cor

In [4]:
food_facts.shape

(356027, 163)

#### 1.1 Data Cleanup

In [ ]:
food_facts.describe(include='all').T

In [ ]:
def count_nan_values(dataset):
    return dataset.apply(lambda x: sum(x.isnull()), axis=0)

count_nan_values(food_facts)

If the majorty of data is missing than interpolation will be highly skewed so we will drop the columns where 70% or more are missing values.

In [ ]:
row_count = food_facts.shape[0]
nan_value_min_threshold = int((70 * row_count) / 100)
food_facts = food_facts.dropna(axis=1,thresh=nan_value_min_threshold)
food_facts.shape

In [ ]:
food_facts.info()

Above result shows that none of the rows is completely NULL. So lets drop the rows where at least 70% of the elements are NaN.

In [ ]:
col_count = food_facts.shape[1]
nan_value_min_threshold = int((70 * col_count) / 100)
food_facts = food_facts.dropna(axis=0,thresh=nan_value_min_threshold)
food_facts.shape

Now we need to specify which nutrients are available to us that we want to measure and a function that could apply other functions on their corresponding columns.

In [ ]:
# For consistency we will raname saturated-fat to saturated_fat
food_facts = food_facts.rename(columns={'saturated-fat_100g': 'saturated_fat_100g'})

MEASURED_NUTRIENTS = ['sugars', 'carbohydrates', 'salt', 'sodium', 'proteins', 'saturated_fat', 'fat']

def apply_nutrients(dataset, transform_func, nutrients=MEASURED_NUTRIENTS):
    result = None
    for nutrient in nutrients:
        col_name = f'{nutrient}_100g'
        result = transform_func(dataset, col_name)
    return result

Next thing we want to do is detect and remove outlier in our data. First we will plot some boxplots per each nutrient to get a better idea where the outliers lie and then we will try to remove them.

In [ ]:
def draw_boxplots_per_nutrient(dataset, col_name):
    plt.figure()
    sns.boxplot(dataset[col_name])
        
apply_nutrients(food_facts, draw_boxplots_per_nutrient)

We can see that proteins have negative values which are obviously not legit so we have to remove them. But just to sure we will apply the same to all the nutrient columns. 

In [ ]:
def remove_negative_values(dataset, col_name):
    dataset.loc[dataset[col_name] < 0, col_name] = 0

apply_nutrients(food_facts, remove_negative_values)
food_facts.shape

Now let's remove outliers from these columns. We know 99% of the data is in the third quartile so we will remove data present outside this range.

In [ ]:
def remove_outliers(dataset, col_name): 
    return dataset[np.abs(dataset[col_name] - dataset[col_name].mean()) <= (3 * dataset[col_name].std())]

food_facts = apply_nutrients(food_facts, remove_outliers)
apply_nutrients(food_facts, draw_boxplots_per_nutrient)
food_facts.shape

Intuitively we know that all these above nutrients are not different for each product so we cannot interpolate them with some preditive model. Best way to do would to fill in mean value for these columns.

In [ ]:
def fill_with_mean_values_where_nan(dataset, col_name):
    dataset[col_name].fillna(dataset[col_name].mean(), inplace=True)

apply_nutrients(food_facts, fill_with_mean_values_where_nan)
food_facts.shape

#### 1.2 Data Preparation

First we need to return mean values as well as number of values. Not all countries have nutrition data (some may have very few entries and there may not be any entries for the nutrient in question) so we set those mean values to `sys.maxsize` so that they can be filtered out later.

In [ ]:
def get_nutrient_info(dataset, country, nutrient):
    col_name = f'{nutrient}_100g'
    country_df = dataset[(dataset['countries_en'] == country)][col_name]
    nutrient_list = list(country_df[country_df.notnull()])     
    entry_count = len(nutrient_list)
    
    if len(nutrient_list) == 0:
        nutrient_mean = sys.maxsize
    else:
        nutrient_mean = np.mean(nutrient_list)
        
    return nutrient_mean, entry_count

There are three features that show us country information: `"countries"`,`"countries_en"` and `"countries_tags"`. We need to decide which feature to use. Unfortunately, there is not very much information regarding the features, so we have to do a bit of guesswork. Let's explore after setting up a dictionary per country where `"countries_en"` are the keys and related `"countries"` are the values:

In [ ]:
# Create a dictionary where "countries_en" are the keys and "countries" are the values
def get_all_country_names(dataset):
    all_country_list = []
    for country in list(dataset['countries_en'].unique()):
        try:
            if len(country.split(',')) == 1:
                all_country_list.append(country)
        except:
            print('Error:', country)

    all_country_dict = {}
    # Add other aliases of a country name, e.g. United States, US, etc.
    for country in all_country_list:
        all_country_dict[country] = list(dataset[dataset['countries_en'] == country]['countries'].unique())
    dataset.countries_tags.unique()
    return all_country_dict

It becomes apparent that `"countries"` is far more variable than `"countries_en"` where `"countries_en"` serves as an aggregation of `"countries"`. As shown below, the United States may be represented as any of the 11 different names in the `"countries"` feature, but it is represented as a whole as *"United States"* in the `"countries_en"` feature. We can imagine how difficult it might be to parse through each entry in `"countries"` to determine what all the different levels refer to, so keeping it high level with `"countries_en"` is in our best interest.

In [ ]:
all_country_dict = get_all_country_names(food_facts)
all_country_dict['United States']

Next we need a function to associate country names with their nutrient consumption per 100g. We will extract average nutrient values per country as well as how many relevent entries have been made by users:

In [ ]:
def collect_nutrient_stats_per_country(dataset, all_country_dict, nutrients=MEASURED_NUTRIENTS):
    nutrient_dict = {}
    for country in list(all_country_dict.keys()):
        nutrient_dict[country] = []

    # Fill in the empty dictionary with all nutrition info and entry counts per country
    for nutrient in nutrients:
        for country in list(all_country_dict.keys()):
            nutrient_dict[country] = nutrient_dict[country] + list(get_nutrient_info(dataset, country, nutrient))
    return nutrient_dict

After building a dictionary with the above mentioned data we can examine, e.g. the entry for the _United States_ where we get all needed numerical values which at this point we do not know which is which.

In [ ]:
nutrient_dict = collect_nutrient_stats_per_country(food_facts, all_country_dict)
nutrient_dict['United States']

Next step is to build a Pandas Dataframe with named features and our aggregated data as observations. We generate a mean value and an entry count for each measured nutrient.

In [ ]:
def create_dataframe_with_features(nutrient_dict, nutrients=MEASURED_NUTRIENTS):
    df_columns = []
    for nutrient in nutrients:
        df_columns = df_columns + [f'avg_{nutrient}',
                                   f'{nutrient}_entry_count']
    
    # Transform the dictionary into a dataframe and display
    nutrient_df = pd.DataFrame.from_dict(nutrient_dict).T
    nutrient_df = nutrient_df.reset_index(drop=False)
    nutrient_df.columns = ['country'] + df_columns
    pd.set_option("display.max_columns", len(nutrient_df.columns))
    
    return nutrient_df

In [ ]:
nutrient_df = create_dataframe_with_features(nutrient_dict)
print(f'Countries count: {len(nutrient_df)}')
nutrient_df.head()

In [ ]:
nutrient_df.country

It is time to do some cleanup. We can notice from the `"country"` column that there are some weird entries. First we want to remove all non English ones and some bad definitions of a country like, e.g. _European Union_, _World_, etc.
Additionally we have to filter those countries with no available nutrition data which we previously marked with `sys.maxsize`.

In [ ]:
def drop_invalid_country_entries(nutrient_df): 
    countries_without_data = np.logical_not((nutrient_df == sys.maxsize).any(1))
    
    drop_countries = [
        'European Union', 
        'Other-japon', 
        'Other-turquie',              
        'World',
        'fr:Quebec', 
        'الإمارات-العربية-المتحدة',
        'السعودية',
        'تونس',
        'سلطنة-عمان'
        'لبنان',
        '香港',
        'भारत'
    ]
    inaccurate_countries = np.logical_not(nutrient_df['country'].isin(drop_countries))

    return nutrient_df[countries_without_data & inaccurate_countries]

In [ ]:
cleaned_nutrient_df = drop_invalid_country_entries(nutrient_df)
print(f'Countries count: {len(cleaned_nutrient_df)}')

We are not done with filtering. Since some countries may dominate the dataset in terms of number of uploads, we have to be aware of the fact that aggregated information from countries with fewer entries may not be as accurate as countries with many entries. For example, a country with 10 000 entries and a mean sugar value per 100g of 20g is probably more reliable than a country with 50 entries and a mean sugar value per 100g of 40g, and the latter's true mean sugar content may not actually be much higher than that of the former. So we need to define an entry count threshold that we think should provide enough data to give accurate results. Based on some experimentation I chose 150 entries as our threshold.

In [ ]:
def filter_above_nutrient_entry_count_threshold(dataset, entry_count_threshold, nutrients=MEASURED_NUTRIENTS):
    filter_query = ' & '.join([f'{nutrient}_entry_count > {entry_count_threshold}' for nutrient in nutrients])
    print(f'FILTER QUERY: {filter_query}')
    return dataset.query(filter_query)

filtered_nutrient_df = filter_above_nutrient_entry_count_threshold(cleaned_nutrient_df, 150)
print(f'Countries count {len(filtered_nutrient_df)}')
filtered_nutrient_df

#### 1.3 Data Visualization

In [ ]:
def plot_nutrient_consumption_per_country(dataset, nutrient):
    x_pos = np.arange(len(dataset.country))
    plt.barh(x_pos, dataset[f'avg_{nutrient}'], align='center', alpha=0.5)
    plt.title(f'Average total {nutrient} content per 100g')
    plt.yticks(x_pos, dataset.country, fontsize = 12)
    plt.xlabel(f'{nutrient.title()}/100g')
    plt.show()

for nutrient in MEASURED_NUTRIENTS:
    plot_nutrient_consumption_per_country(filtered_nutrient_df, nutrient)

In [ ]:
def get_country_with_max_content_per_nutrient(dataset, nutrient):
    avg_nutrient_col_name = f'avg_{nutrient}'
    x = max(dataset[avg_nutrient_col_name])
    filtered_dataset = dataset[(dataset[avg_nutrient_col_name] == x)]
    return filtered_dataset[['country', avg_nutrient_col_name]]

for nutrient in MEASURED_NUTRIENTS:
    df = get_country_with_max_content_per_nutrient(filtered_nutrient_df, nutrient)
    print(f'Top {nutrient.title()}/100g consuming country')
    display(HTML(df.to_html(index=False)))

Now we would like to dig into some correlations between all numerical features in order to find someting interesting. We would do that with a heatmap using Spearman's rank correlation coefficient:

In [ ]:
def plot_nutrient_correlations(dataset):
    cmap = sns.diverging_palette(0, 255, sep=1, n=256, as_cmap=True)

    correlations = dataset[['additives_n','ingredients_from_palm_oil_n','ingredients_that_may_be_from_palm_oil_n','energy_100g','fat_100g',
                       'saturated_fat_100g','carbohydrates_100g','sugars_100g','proteins_100g','salt_100g','sodium_100g']].corr()
    # Plot preparation
    plt.figure(figsize=(12,12))
    plt.title('Correlation between food preferences', y=1.01, size=20)
    sns.heatmap(
            correlations.corr(method='spearman'), 
            linewidths=0.5, 
            vmax=1.0, 
            square=True, 
            center=0,
            cmap=cmap, 
            annot=True, 
            cbar_kws={"shrink": .75, "orientation": "horizontal"}
        )

plot_nutrient_correlations(food_facts)

It is clear from above result that `"salt_100g"` is highly correlated to `"sodium_100g"`. This is very obvious and also justified by data. 

### 2. Food choices and preferences of college students dataset

This dataset includes information on food choices, nutrition, preferences, childhood favorites, and other information from college students. The goal is to take a look at student's nutrition habits, e.g.:
* What does influence on students cooking frequency?
* Are students who are active in sports put more attention to nutritional check than the others?
* Is there any correlation between types of world cuisines that students like or do not like?


In [ ]:
food_choices_data = pd.read_csv('data/food_coded.csv')
food_choices_data.head()

In [ ]:
food_choices_data.shape

In [ ]:
food_choices_data.describe(include='all').T

#### 2.1. Data Cleanup

`GPA` and `weight` features have many unique values and the highest values of 'top' column are numbers. It looks like their dtype might be wrong and they can contain blended values.

In [ ]:
def get_column_dtypes(dataset):
    return np.array([dataset[x].dtype for x in dataset.columns])

col_dtypes = get_column_dtypes(food_choices_data)

for i, j in Counter(col_dtypes).items():
    print('dtype: ', i, ' -> ', 'column_count: ', j)

Now we need to inspect the object datatype columns for blended values:

In [ ]:
def get_object_dtype_columns(dataset):
    dtype_df = pd.DataFrame({'dtype_': col_dtypes}, index=dataset.columns)
    return dtype_df[dtype_df['dtype_'] == object]

obj_cols_df = get_object_dtype_columns(food_choices_data)
obj_cols_df

Next we want to be able to really find out the datatype of each feature value:

In [ ]:
def extract_real_feature_value_types(dataset, dtype_df):
    dtypes = {}

    for feature in dtype_df.index.values:
        feat_dict = {}

        for value in dataset[feature].values:
            # Get real datatype for the current feature value
            dtype = str(type(value))
            match = re.search('int|float|str', dtype)

            # Create a dict with number of dtypes for particular feature
            if match.group() not in feat_dict.keys():
                feat_dict[match.group()] = 1
            else:
                feat_dict[match.group()] += 1
        dtypes[feature] = feat_dict
        # Clean up the dict before next iteration
        feat_dict = {}
    return dtypes

dtypes_dict = extract_real_feature_value_types(food_choices_data, obj_cols_df)

# Create transposed data frame with dtypes counter for each object feature
dtype_value_count_df = pd.DataFrame.from_dict(dtypes_dict).T
dtype_value_count_df

`'GPA'` and `'weight'` features have mostly string values with a few NaN's. It is a good news for us, because it will be easier to isolate the digits with regex.

In [ ]:
food_choices_data['GPA'].unique()

As we can see above, there is a blended value and some cells without digits. Here we got 2 float values which are NaN's what we could see in features dtype table in one of previous subsections. We will fix that and replace incorrect and missing values with the most common feature's value. It should be fine since the values distribution does not emphasize one of the unique values in significant way.

In [ ]:
def remove_gpa_col_blended_values(dataset):
    most_common_val = dataset['GPA'].value_counts().head(1).index[0]
    # Use regex to clean blended data, fill missing values and set up dtype
    return dataset['GPA'].str.replace(r'[^\d\.\d+]', '').replace((np.nan, ''), most_common_val).astype(float).round(2)

food_choices_data['GPA'] = remove_gpa_col_blended_values(food_choices_data)

In [ ]:
food_choices_data['GPA'].unique()

In [ ]:
food_choices_data['weight'].unique()

In [ ]:
def remove_weight_col_blended_values(dataset):
    # Clean blended values, non-numeric become NaN
    return dataset['weight'].str.replace(r'[^\d\d\d]', '').replace('', np.nan).astype(float)

food_choices_data['weight'] = remove_weight_col_blended_values(food_choices_data)

In [ ]:
food_choices_data['weight'].unique()

#### 2.2. Data Preparation and Visualization

In [ ]:
def plot_weight_distributions_per_gender(dataset):
    fig, ax = plt.subplots(figsize=[16,4])

    # Create two distributions for both genders
    gender_dict = {1: 0, 2: 1}
    for gen, frame in dataset[['Gender', 'weight']].dropna().groupby('Gender'):
        sns.distplot(frame['weight'], ax=ax, label=['Female', 'Male'][gender_dict[gen]])

    ax.set_title('Weight distributions per gender')
    ax.legend()
    
plot_weight_distributions_per_gender(food_choices_data)

In [ ]:
def prepare_category_data(data, x_var, hue_var, hue_class, x_names=[], hue_names=[]):
    # Prepare dichotomous and/or ordinal variable/s to graphic representation
    
    # Create data frame
    df = pd.DataFrame(index=x_names)
    
    # Prepare converted values for each hue
    for i, j in [(name, ind + 1) for ind, name in enumerate(hue_names)]:
        df[i] = data[x_var][data[hue_var] == j].value_counts().sort_index().values
        df[i] = ((df[i] / df[i].sum()) * 100).round(1)
        
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'categories'}, inplace=True)
    
    return pd.melt(df, id_vars="categories", var_name=hue_class, value_name="percent")

In [ ]:
def plot_cooking_frequency_per_gender(dataset):
    # New category names
    labels = ['daily', '3 times/w', 'rarely','holidays', 'never']
    
    # Data preparation for plotting
    df_cook_sex = prepare_category_data(
        data=dataset, 
        x_var='cook', 
        x_names=labels,
        hue_var='Gender', 
        hue_class='sex', 
        hue_names=['Male', 'Female']
    )

    with sns.plotting_context(context='notebook', font_scale=1.25):
        sns.catplot(
            x='categories', 
            y='percent', 
            hue='sex', 
            data=df_cook_sex,            
            kind='bar', 
            palette='cubehelix', 
            height=8
        )
        # Plot labels
        plt.xlabel('Frequency', fontsize=16)
        plt.ylabel('Percent of Female/Male [%]', fontsize=16)
        plt.title('Cooking frequency', y=1.01, size=25)
        
plot_cooking_frequency_per_gender(food_choices_data)

Distribution differences between both genders are significant. We can see that almost half of females cooks rarely. Cooking is more frequent on all levels in comparison to males. 'Holidays' and 'never' categories are rather opposite to cooking in the academic year where males have full dominance over females.

In [ ]:
def plot_income_distribution_per_gender(dataset):
    # New category names
    labels = ['very low', 'low', 'average','above-average', 'high', 'very high']
    # Data preparation for plotting
    df_inc = prepare_category_data(
        data=dataset, 
        x_var='income', 
        x_names=labels,
        hue_var='Gender', 
        hue_class='sex',  
        hue_names=['Female', 'Male']
    )

    with sns.plotting_context(context='notebook', font_scale=1.25):
        # Seaborn's barplot creator
        sns.catplot(
            x='categories', 
            y='percent', 
            hue='sex', 
            data=df_inc,           
            kind='bar', 
            palette='Blues', 
            height=8
        )
        # Plot labels
        plt.xlabel('Income', fontsize=15)
        plt.ylabel('Percent of Female/Male [%]', fontsize=15)
        plt.title('Income distribution', y=1.01, size=25) 
        
plot_income_distribution_per_gender(food_choices_data)

Income distribution can in some way explain why males tends to cook less. More than a half people defined their family income as high or very high.

In [ ]:
def plot_nutritional_check_frequency_per_sports_activity(dataset):
    # New category names
    labels = ['never', 'rare', 'sometimes', 'often', 'always']
    
    # Data preparation for plotting
    df_inc = prepare_category_data(
        data=dataset, 
        x_var='nutritional_check', 
        x_names=labels,
        hue_var='sports', 
        hue_class='activity',  
        hue_names=['no sport', 'sport']
    )

    with sns.plotting_context(context='notebook', font_scale=1.25):
        sns.catplot(
            x='categories', 
            y='percent', 
            hue='activity', 
            data=df_inc,           
            kind='bar', 
            palette='muted', 
            height=8
        )
        # Plot labels
        plt.xlabel('Frequency', fontsize=16)
        plt.ylabel('Percent dependent of activity [%]', fontsize=16)
        plt.title('Nutritional check frequency', y=1.01, size=25) 

plot_nutritional_check_frequency_per_sports_activity(food_choices_data)

The plot above explores how nutritional check frequency is connected with sport activity. The conclusion is that sports oriented people do more frequent nutritional checks. 

In [ ]:
def prepare_data_on_off_campus(dataset):
    # New category names
    labels = ['daily', '3 times/w', 'rarely','holidays', 'never']
    # Create data frame
    df_camp = pd.DataFrame(index=labels)

    # Add new columns in df_camp with data
    df_camp['on_campus'] = dataset['cook'][dataset['on_off_campus'] == 1].value_counts().sort_index().values
    # Values != 1 are different types of accommodation outside the campus
    df_camp['off_campus'] = dataset['cook'][dataset['on_off_campus'] > 1].value_counts().sort_index().values

    # Prepare converted values for each hue
    df_camp['on_campus'] = ((df_camp['on_campus'] / df_camp['on_campus'].sum()) * 100).round(1)
    df_camp['off_campus'] = ((df_camp['off_campus'] / df_camp['off_campus'].sum()) * 100).round(1)

    df_camp.reset_index(inplace=True)
    df_camp.rename(columns={'index': 'categories'}, inplace=True)

    # Reshaping data frame
    return pd.melt(df_camp, id_vars="categories", var_name='on/off campus', value_name="percent")

In [ ]:
def plot_cooking_frequency_when_on_or_off_campus(dataset):
    df_camp = prepare_data_on_off_campus(dataset)
    with sns.plotting_context(context='notebook', font_scale=1.25):
        sns.catplot(
            x='categories', 
            y='percent', 
            hue='on/off campus', 
            data=df_camp, 
            kind='bar', 
            palette='GnBu_d', 
            height=8
        )
        # Plot labels
        plt.xlabel('Frequency', fontsize=16)
        plt.ylabel('Percent of on/off campus [%]', fontsize=16)
        plt.title('Cooking frequency', y=1.01, size=25)
    
plot_cooking_frequency_when_on_or_off_campus(food_choices_data)


This next feature explores cooking frequency dependent on the location, being in this case on or off campus. Clearly people who live off the campus tend to cook often.

In [ ]:
def plot_world_cuisine_preferences(dataset):
    # New category names
    food = ['greek_food', 'indian_food', 'italian_food', 'thai_food', 'persian_food', 'ethnic_food']
    # Set diverging palette
    cmap = sns.diverging_palette(0, 255, sep=1, n=256, as_cmap=True)
    # Plot preparation
    plt.figure(figsize=(12,12))
    plt.title('Correlation between food preferences', y=1.01, size=20)
    sns.heatmap(
        dataset[food].corr(method='spearman'), 
        linewidths=0.5, 
        vmax=1.0, 
        square=True, 
        center=0,
        cmap=cmap, 
        annot=True, 
        cbar_kws={"shrink": .75, "orientation": "horizontal"}
    )


plot_world_cuisine_preferences(food_choices_data)

The heatmap above shows us correlations between different types of world cuisines. Surveyed people weighted their preferences in 1-5 point scale. The correlation between these variables is very strong. We can conclude that people have similar taste when it comes to types of world cuisines. The only one feature that is not dominant is `'italian_food'`.

### 3. Nutrition Facts for McDonalds Menu dataset

In [ ]:
mcdonalds_data = pd.read_csv('data/menu.csv')
pd.set_option("display.max_columns", len(mcdonalds_data.columns))
mcdonalds_data.head()

In [ ]:
print(mcdonalds_data.shape)
mcdonalds_data.describe()

#### 3.1. Data Preparation

In [ ]:
NUTRIENT_CATEGORIES = ['Cholesterol', 'Sugars', 'Sodium', 'Carbohydrates', 'Protein', 'Saturated Fat', 'Trans Fat', 'Total Fat', 'Dietary Fiber']

In [ ]:
def get_menu_categories_per_max_nutrient_content(dataset, nutrient):
    x = max(dataset[nutrient])
    filtered_dataset = dataset[(dataset[nutrient] == x)]
    return filtered_dataset[['Category', 'Item', nutrient]]

for nutrient in NUTRIENT_CATEGORIES:
    df = get_menu_categories_per_max_nutrient_content(mcdonalds_data, nutrient)
    print(f'Menu items and menu categories containing max {nutrient} levels')
    display(HTML(df.to_html(index=False)))

#### 3.2. Data Visualization

In [ ]:
mcdonalds_data.groupby('Category')['Item'].count().plot(kind='bar')

In [ ]:
for nutrient in NUTRIENT_CATEGORIES:   
    plot = sns.violinplot(x="Category", y=nutrient, data=mcdonalds_data)
    plt.xticks(rotation=90, size=12)
    plt.title(nutrient)
    plt.show()

### Conclusion
With this analysis we tried to detect eating habbits of college students, nutrient consumption across different countries and nutrient contents of McDonald's menu. 3 datasets were used:
* Open Food Facts 
* Food choices and preferences of college students
* Nutrition Facts for McDonalds Menu

After performing data cleanup we were able to transform the data in different forms, do meaningful aggregations and regroupings and finally plot it in various ways and forms in order to get rich insights about it.

### References
[1] Pandas, "Pandas Documentation", https://pandas.pydata.org/pandas-docs/stable/

[2] Seaborn, "Seaborn Documentation", https://seaborn.pydata.org/

[3] Wikipedia, "Spearman's rank correlation coefficient", https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

[4] Wikipedia, "Violin Plot", https://en.wikipedia.org/wiki/Violin_plot